# MARATONA BEHIND THE CODE 2020

## DESAFIO 2: PARTE 2

### Introdução

Na parte 1 deste desafio, você realizou o pré-processamento e o treinamento de um modelo a partir de um conjunto de dados base fornecido. Nesta segunda etapa você irá integrar todas as transformações e eventos de treinamento criados anteriormente em uma Pipeline completa para *deploy* no **Watson Machine Learning**!

### Preparação do Notebook

Primeiro realizaremos a instalação do scikit-learn e a importação das mesmas bibliotecas utilizadas anteriormente

In [ ]:
!pip install scikit-learn==0.20.0 --upgrade

In [ ]:
import json
import requests
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, cross_validate

É necessário inserir o conjunto de dados base novamente como um dataframe pandas, seguindo as instruções

![alt text](https://i.imgur.com/K1DwL9I.png "importing-csv-as-df")

Após a seleção da opção **"Insert to code"**, a célula abaixo será preenchida com o código necessário para importação e leitura dos dados no arquivo .csv como um DataFrame Pandas.

In [ ]:

import types
import pandas as pd

## Removed the "body" variable, so you could adapt

df_data_1 = pd.read_csv(body)
df_data_1.head()


### Construção da Pipeline completa para encapsulamento no WML

#### Preparando transformações personalizadas para carregamento no WML

Na etapa anterior, foi mostrado como criar uma transformação personalizada, através da declaração de uma classe Python com os métodos ``fit`` e ``transform``.

    - Código da transformação personalizada DropColumns():
    
    from sklearn.base import BaseEstimator, TransformerMixin
    # All sklearn Transforms must have the `transform` and `fit` methods
    class DropColumns(BaseEstimator, TransformerMixin):
        def __init__(self, columns):
            self.columns = columns
        def fit(self, X, y=None):
            return self
        def transform(self, X):
            # Primeiro realizamos a cópia do dataframe 'X' de entrada
            data = X.copy()
            # Retornamos um novo dataframe sem as colunas indesejadas
            return data.drop(labels=self.columns, axis='columns')

Para integrar esses tipos de transformações personalizadas nas Pipelines do Watson Machine Learning, é necessário primeiramente empacotar seu código personalizado como uma biblioteca Python. Isso pode ser feito facilmente com o uso da ferramenta *setuptools*.

No seguinte repositório git: https://github.com/vnderlev/sklearn_transforms temos todos os arquivos necessários para a criação de um pacote Python, nomeado **my_custom_sklearn_transforms**.
Esse pacote possui a seguinte estrutura de arquivos:

    /my_custom_sklearn_transforms.egg-info
        dependency_links.txt
        not-zip-safe
        PKG-INFO
        SOURCES.txt
        top_level.txt
    /my_custom_sklearn_transforms
        __init__.py
        sklearn_transformers.py
    PKG-INFO
    README.md
    setup.cfg
    setup.py
    
O arquivo principal, que irá conter o código das nossas transformadas personalizadas, é o arquivo **/my_custom_sklearn_transforms/sklearn_transformers.py**. Se você acessá-lo no repositório, irá notar que ele contém exatamente o mesmo código declarado na primeira etapa (a classe DropColumns).

Caso você tenha declarado transformações próprias (além da DropColumn fornecida), você deverá adicionar todas as classes dessas transformadas criadas por você nesse mesmo arquivo. Para tal, você deve realizar o fork desse repositório (isso pode ser feito na própria interface Web do Github, clicando no botão conforme a imagem abaixo), e adicionar suas classes personalizadas no arquivo **sklearn_transformers.py**.

![alt text](https://i.imgur.com/D81E1uM.png "forking-a-repo")

Se você somente fez o uso da transformação fornecida (DropColumns), pode ignorar essa etapa de fork, e seguir utilizando o pacote base fornecido! :)

Após a preparação do seu pacote Python com as suas transformadas personalizadas, substitua o link do repositório git na célula abaixo e execute-a. Caso você não tenha preparado nenhuma nova transformada, execute a célula com o link do repositório já fornecido. 

<hr>
    
**OBSERVAÇÃO**

Caso a execução da célula abaixo retorne um erro de que o repositório já existe, execute:

**!rm -r -f sklearn_transforms**

In [28]:
# substitua o link abaixo pelo link do seu repositório git (se for o caso)
!git clone https://github.com/vnderlev/sklearn_transforms.git

fatal: destination path 'sklearn_transforms' already exists and is not an empty directory.


In [29]:
!cd sklearn_transforms
!ls -ltr

total 68
drwxr-x--- 5 dsxuser dsxuser  4096 Sep  1 00:22 sklearn_transforms
-rw-r----- 1 dsxuser dsxuser 62139 Sep  1 00:22 sklearn_transforms.zip


Para subir o código no WML, precisamos enviar um arquivo .zip com todo o código fonte, então iremos zipar o diretório clonado em seguida:

In [30]:
!zip -r sklearn_transforms.zip sklearn_transforms

updating: sklearn_transforms/ (stored 0%)
updating: sklearn_transforms/setup.py (deflated 46%)
updating: sklearn_transforms/my_custom_sklearn_transforms.egg-info/ (stored 0%)
updating: sklearn_transforms/my_custom_sklearn_transforms.egg-info/SOURCES.txt (deflated 48%)
updating: sklearn_transforms/my_custom_sklearn_transforms.egg-info/PKG-INFO (deflated 33%)
updating: sklearn_transforms/my_custom_sklearn_transforms.egg-info/top_level.txt (stored 0%)
updating: sklearn_transforms/my_custom_sklearn_transforms.egg-info/not-zip-safe (stored 0%)
updating: sklearn_transforms/my_custom_sklearn_transforms.egg-info/dependency_links.txt (stored 0%)
updating: sklearn_transforms/my_custom_sklearn_transforms/ (stored 0%)
updating: sklearn_transforms/my_custom_sklearn_transforms/__init__.py (stored 0%)
updating: sklearn_transforms/my_custom_sklearn_transforms/sklearn_transformers.py (deflated 46%)
updating: sklearn_transforms/PKG-INFO (deflated 31%)
updating: sklearn_transforms/README.md (deflated 15%

Com o arquivo zip do nosso pacote carregado no Kernel deste notebook, podemos utilizar a ferramenta pip para instalá-lo, conforme a célula abaixo:

In [31]:
!pip install sklearn_transforms.zip

Processing ./sklearn_transforms.zip
  Stored in directory: /home/dsxuser/.tmp/pip-ephem-wheel-cache-l7ui49ij/wheels/8f/88/32/f886e7510a37b111e2a1b7e689e04450acda46732970a7ed78
Successfully built my-custom-sklearn-transforms
  Found existing installation: my-custom-sklearn-transforms 1.0
    Uninstalling my-custom-sklearn-transforms-1.0:
      Successfully uninstalled my-custom-sklearn-transforms-1.0


Podemos agora realizar a importação do nosso pacote personalizado em nosso notabook!

Iremos importar a transformação DropColumns. Se você possui outras transformações personalizadas, não se esqueça de importá-las!

In [32]:
from my_custom_sklearn_transforms.sklearn_transformers import DropColumns
from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier

#### Declarando a Pipeline

Após a importação das transformações personalizadas como um pacote Python, podemos partir para a declaração da nossa Pipeline.

O processo é bem semelhante ao realizado na primeira etapa, porém com algumas diferenças importantes, então preste bem atenção!

A Pipeline exemplo possui três estágios: 

    - remover a coluna "NOME"
    - imputar "zeros" em todos os valores faltantes
    - inserir os dados pré-processados como entrada em um modelo treinado
    
Relembrando, a entrada desta Pipeline será o conjunto cru de dados fornecido exceto a coluna "LABELS" (variável-alvo a ser determinada pelo modelo).

Teremos então 17 valores de entrada **na PIPELINE** (no modelo serão 16 entradas, pois a coluna NAME será removida no primeiro estágio após a transformação DropColumn).

    MATRICULA       - número de quatro algarismos único para cada estudante
    NOME            - nome completo do estudante
    FALTAS_DE       - número de faltas na disciplina de ``Direito Empresarial``
    FALTAS_EM       - número de faltas na disciplina de ``Empreendedorismo``
    FALTAS_MF       - número de faltas na disciplina de ``Matemática Financeira``
    MEDIA_DE        - média simples das notas do aluno na disciplina de ``Direito Empresarial`` (0-10)
    MEDIA_EM        - média simples das notas do aluno na disciplina de ``Empreendedorismo`` (0-10)
    MEDIA_MF        - média simples das notas do aluno na disciplina de ``Matemática Financeira`` (0-10)
    HRS_ESTUDO_DE   - horas de estudo particular na disciplina de ``Direito Empresarial``
    HRS_ESTUDO_EM   - horas de estudo particular na disciplina de ``Empreendedorismo``
    HRS_ESTUDO_MF   - horas de estudo particular na disciplina de ``Matemática Financeira``
    REPROVACOES_DE  - número de reprovações na disciplina de ``Direito Empresarial``
    REPROVACOES_EM  - número de reprovações na disciplina de ``Empreendedorismo``
    REPROVACOES_MF  - número de reprovações na disciplina de ``Matemática Financeira``
    LIVROS_TEXTO    - quantidade de livros e textos acessados pelo aluno no sistema da universidade
    AULAS_AO_VIVO   - horas de aulas ao vivo presenciadas pelo aluno (total em todas as disciplinas)
    EXERCICIOS      - número de exercícios realizados pelo estudante (total em todas as disciplinas) no sistema da universidade

A saída da Pipeline será um valor estimado para a coluna "LABELS".

In [33]:
# Criação de uma Transform personalizada ``DropColumns``

rm_columns = DropColumns(
    columns=["MATRICULA","NOME"]
)

In [34]:
# Criação de um objeto ``SimpleImputer``

si = SimpleImputer(
    missing_values=np.nan,  # os valores faltantes são do tipo ``np.nan`` (padrão Pandas)
    strategy='most_frequent',  # a estratégia escolhida é a alteração do valor faltante por uma constante
)

In [35]:
# Balanceamento

# Colocando as notas maiores que 10 como 10 para balancear
df_data_1.loc[df_data_1["NOTA_MF"] > 10, "NOTA_MF"] = 10
df_data_1.loc[df_data_1["NOTA_GO"] > 10, "NOTA_GO"] = 10
df_data_1.loc[df_data_1["NOTA_EM"] > 10, "NOTA_EM"] = 10
df_data_1.loc[df_data_1["NOTA_DE"] > 10, "NOTA_DE"] = 10

In [36]:
df_data_1.head()

,MATRICULA,NOME,REPROVACOES_DE,REPROVACOES_EM,REPROVACOES_MF,REPROVACOES_GO,NOTA_DE,NOTA_EM,NOTA_MF,NOTA_GO,INGLES,H_AULA_PRES,TAREFAS_ONLINE,FALTAS,PERFIL
0,502375,Márcia Illiglener,0,0,0,0,6.2,5.8,4.6,5.9,0.0,2,4,3,EXATAS
1,397093,Jason Jytereoman Izoimum,0,0,0,0,6.0,6.2,5.2,4.5,1.0,2,4,3,EXATAS
2,915288,Bartolomeu Inácio da Gama,0,0,0,0,7.3,6.7,7.1,7.2,0.0,5,0,3,HUMANAS
3,192652,Fernanda Guedes,1,3,1,1,0.0,0.0,0.0,0.0,1.0,4,4,4,DIFICULDADE
4,949491,Alessandre Borba Gomes,1,3,1,1,0.0,0.0,0.0,0.0,1.0,5,2,5,DIFICULDADE


In [57]:
# Definição das colunas que serão features (nota-se que a coluna MATRICULA e NOME não está presente)
features = [
    'REPROVACOES_DE', 'REPROVACOES_EM', "REPROVACOES_MF", "REPROVACOES_GO",
    "NOTA_DE", "NOTA_EM", "NOTA_MF", "NOTA_GO",
    "INGLES", "H_AULA_PRES", "NOME", "MATRICULA", "TAREFAS_ONLINE", "FALTAS", 
]

# Definição da variável-alvo
target = ["PERFIL"]

# Preparação dos argumentos para os métodos da biblioteca ``scikit-learn``
X = df_data_1[features]
y = df_data_1[target]

**ATENÇÃO!!**

A célula acima, embora muito parecida com a definição de features na primeira etapa deste desafio, possui uma grande diferença!

Nela está presente a coluna "NOME" como uma feature! Isso ocorre pois neste caso essas são as entradas da *PIPELINE*, e não do modelo.

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=337, shuffle=False)

Na célula abaixo é realizada a declaração de um objeto **Pipeline** do scikit-learn, onde é declarado o parâmetro *steps*, que nada mais é do que uma lista com as etapas da nossa pipeline:

    'remove_cols'     - transformação personalizada DropColumns
    'imputer'         - transformação embutida do scikit-learn para imputação de valores faltantes
    'dtc'             - um classificador via árvore de decisão
    
Note que passamos como passos as transformadas instanciadas anteriormente, sob nome `rm_columns` e `si`.

In [59]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix

# Criação da nossa pipeline para armazenamento no Watson Machine Learning:
my_pipeline = Pipeline(
    steps=[
        ('remove_cols', rm_columns),
        ('imputer', si),
        ('rfc', OneVsRestClassifier(XGBClassifier(max_depth=4)))
    ]
)

Em seguida iremos executar o método `fit()` da Pipeline, realizando o pré-processamento e o treinamento do modelo de uma só vez.

In [60]:
# Inicialização da Pipeline (pré-processamento e realização do treinamento do modelo)
my_pipeline.fit(X_train, y_train.values.ravel())

Pipeline(memory=None,
     steps=[('remove_cols', DropColumns(columns=['MATRICULA', 'NOME'])), ('imputer', SimpleImputer(copy=True, fill_value=None, missing_values=nan,
       strategy='most_frequent', verbose=0)), ('rfc', OneVsRestClassifier(estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
     ..._lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
          n_jobs=None))])

Agora que temos uma pipeline completa, com etapas de pré-processamento configuradas e também um modelo por árvore de decisão já treinado, podemos realizar a integração com o Watson Machine Learning!

### Encapsulando uma Pipeline personalizada no Watson Machine Learning

#### Estabelecendo conexão entre o cliente Python do WML e a sua instância do serviço na nuvem

In [1]:
# Biblioteca Python com implementação de um cliente HTTP para a API do WML
from watson_machine_learning_client import WatsonMachineLearningAPIClient

2020-09-01 01:07:01,669 - watson_machine_learning_client.wml_client_error - WARNING - Artifact with artifact_uid: 'f7822e96-79ec-4f17-9f88-f1593e41e728' does not exist.
2020-09-01 01:08:23,397 - watson_machine_learning_client.wml_client_error - WARNING - Artifact with artifact_uid: '42db1688-82e0-467e-bad0-53f38dc54929' does not exist.
2020-09-01 01:08:39,038 - watson_machine_learning_client.wml_client_error - WARNING - Failure during getting all runtimes. (GET https://eu-de.ml.cloud.ibm.com/v4/runtimes?limit=1000)
Status code: 404, body: {"trace":"8b620b22a1fcdec1cd81dc59ff978367","errors":[{"code":"not_found","message":"Requested object ce545f51-3707-453c-bacb-049fae1bee8f could not be found."}]}
2020-09-01 01:08:39,044 - watson_machine_learning_client.repository - ERROR - Failure during getting all runtimes. (GET https://eu-de.ml.cloud.ibm.com/v4/runtimes?limit=1000)
Status code: 404, body: {"trace":"8b620b22a1fcdec1cd81dc59ff978367","errors":[{"code":"not_found","message":"Requeste

As próximas células irão realizar o deploy da pipeline declarada neste notebook no WML. Só prossiga se você já está satisfeito com seu modelo e acha que já é a hora de fazer o deploy da sua solução.

Cole as credenciais de sua instância do Watson Machine Learning na variável na célula abaixo.

É importante que a variável que contém os valores tenha o nome de ``wml_credentials`` para que as próximas células deste notebook executem corretamente.

In [2]:
wml_credentials = {
  "<you can just copy the wml credentials, but wml doesn't work like that anymore>"
}

In [3]:
# Instanciando um objeto cliente do Watson Machine Learning a partir das credenciais fornecidas

clientWML = WatsonMachineLearningAPIClient(wml_credentials)

**ATENÇÃO!!**

Fique atento para os limites de consumo de sua instância do Watson Machine Learning!

Caso você expire a camada grátis, não será possível avaliar seu modelo (pois é necessária a realização de algumas chamadas de API que consomem predições!)

#### Listando todos os artefatos armazenados no seu WML

Para listar todos os artefatos armazenados em seu Watson Machine Learning, você pode usar a seguinte função:

    clientWML.repository.list()

In [75]:
result = clientWML.deployments.score(
    model_endpoint_url,
    scoring_payload
)

print("\n Resultados:")
print(json.dumps(result, indent=4))


 Resultados:
{
    "fields": [
        "prediction",
        "probability"
    ],
    "values": [
        [
            "DIFICULDADE",
            [
                0.9986030459403992,
                0.0010330029763281345,
                6.002801819704473e-05,
                8.979632548289374e-05,
                0.00021394914074335247
            ]
        ]
    ]
}


<hr>

## Parabéns! 

Se tudo foi executado sem erros, você já tem um classificador baseado em machine learning encapsulado como uma API REST!

Para testar a sua solução integrada com um assistente virtual e realizar a submissão, acesse a página:

https://uninassau.maratona.dev

Você irá precisar da endpoint url do seu modelo e das credenciais do WML :)